In [1]:
import geopandas as gpd
import pandas as pd
import numpy as np
from geopy.distance import geodesic
import math

import gurobipy as gp
from gurobipy import GRB

In [2]:
zones = gpd.read_file('taxi_zones/taxi_zones.shp')
table = pd.read_csv('taxi+_zone_lookup.csv')
zones = zones.to_crs("EPSG:4326")

In [3]:
def dist(pt1, pt2):
    return geodesic(pt1, pt2).miles

def optql(x_list, pi_list, dQ, epsilon=0.5):
    '''
    input: x_list: list of geographic coordinates
           pi_list: probability distribution for x_list
           dQ: distance metric
           epsilon: desired privacy level
    
    output: matrix: stochastic transition matrix
            pre_prob: normalized pre-process probability distribution
            post_prob: post-process probability distribution
    '''
    
    pre_prob = np.array(pi_list) / sum(pi_list) # normalize probability distribution
    n = len(x_list) # get number of elements
    threshold = math.exp(epsilon)
    
    # define a model
    model = gp.Model('OptQL')
    
    # add variables accessed as (0, 0), (0, 1), (1, 1), ...
    variables = model.addVars(n, n, lb=0.0, ub=1.0, name='k')
    
    # set objective function
    model.setObjective(gp.quicksum(pre_prob[i] * variables[i, j] * dQ(x_list[i], x_list[j]) \
                                   for i in range(n) for j in range(n)), GRB.MINIMIZE)
    
    # add constraints (1)
    model.addConstrs(variables[i, k] <= pow(threshold, dQ(x_list[i], x_list[j])) * variables[j, k] \
                     for i in range(n) for j in range(n) for k in range(n))
    
    # add constraints (2)
    model.addConstrs(gp.quicksum(variables.select(i, '*')) == 1 for i in range(n))
    
    # constriants (3) are already satisfied
    
    # optimize the model
    model.optimize()
    
    # build a matrix to store the stochastic matrix
    variables = model.getAttr('x', variables)
    matrix = np.zeros((n, n))
    for key, value in variables.items():
        matrix[key] = value
    
    # get post-process probability distribution
    post_prob = pre_prob @ matrix
    
    
    return matrix, pre_prob, post_prob

In [ ]:
zones['Centroid'] = zones['geometry'].apply(lambda x: x.centroid)
x_list = [(lon, lat) for (lon, lat) in zones['Centroid'].apply(lambda x: (x.xy[1][0], x.xy[0][0]))]
pi_list = [geometry.area for geometry in zones['geometry']]

dQ = dist
epsilon = 0.5

p_matrix, pre_prob, post_prob = optql(x_list, pi_list, dQ, epsilon=epsilon)

Academic license - for non-commercial use only - expires 2021-08-07
Using license file /Users/aoqu/gurobi.lic
Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 18191710 rows, 69169 columns and 36313725 nonzeros
Model fingerprint: 0xcbe6a767
Coefficient statistics:
  Matrix range     [1e+00, 1e+07]
  Objective range  [3e-05, 7e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]

Concurrent LP optimizer: dual simplex and barrier
Showing barrier log only...

Presolve removed 69169 rows and 0 columns (presolve time = 5s) ...
Presolve removed 69169 rows and 0 columns (presolve time = 17s) ...
Presolve removed 69169 rows and 0 columns (presolve time = 22s) ...
Presolve removed 69169 rows and 0 columns (presolve time = 26s) ...
Presolve removed 69169 rows and 0 columns
Presolve removed 0 rows and 69169 columns (presolve time = 9s) ...
Presolve removed 0 rows and 69169 co

  87   2.87582787e+00  2.94088604e+00  3.06e-15 5.09e-11  3.58e-09   695s
  88   2.87757602e+00  2.93674345e+00  5.25e-15 5.09e-11  3.25e-09   702s
  89   2.87990172e+00  2.93468041e+00  3.07e-15 4.00e-11  3.01e-09   710s
  90   2.88220242e+00  2.93275138e+00  1.84e-15 3.27e-11  2.78e-09   717s
  91   2.88355421e+00  2.93116917e+00  2.79e-15 4.00e-11  2.62e-09   724s
  92   2.88505664e+00  2.92868871e+00  3.69e-15 5.82e-11  2.40e-09   732s
  93   2.88633136e+00  2.92636886e+00  3.09e-15 4.37e-11  2.20e-09   739s
  94   2.88761114e+00  2.92409666e+00  3.20e-15 6.00e-11  2.01e-09   747s
  95   2.88837008e+00  2.92145704e+00  3.24e-15 6.91e-11  1.82e-09   755s
  96   2.88931108e+00  2.92035321e+00  3.64e-15 6.55e-11  1.71e-09   761s
  97   2.89023201e+00  2.91904545e+00  3.67e-15 9.09e-11  1.58e-09   768s
  98   2.89089454e+00  2.91798694e+00  3.61e-15 1.20e-10  1.49e-09   776s
  99   2.89144594e+00  2.91633535e+00  3.59e-15 1.55e-10  1.37e-09   782s
 100   2.89212356e+00  2.91496481e+00 

 198   2.89890985e+00  2.89899612e+00  6.88e-13 1.98e-09  4.74e-12  1525s
 199   2.89891104e+00  2.89899371e+00  1.03e-12 2.08e-09  4.54e-12  1535s
 200   2.89891212e+00  2.89898950e+00  9.51e-13 1.96e-09  4.25e-12  1542s
 201   2.89891312e+00  2.89898248e+00  8.56e-13 1.75e-09  3.81e-12  1552s
 202   2.89891380e+00  2.89897630e+00  8.12e-13 1.55e-09  3.44e-12  1561s
 203   2.89891448e+00  2.89897469e+00  8.07e-13 1.52e-09  3.31e-12  1567s
 204   2.89891515e+00  2.89897114e+00  7.50e-13 1.13e-09  3.08e-12  1576s
 205   2.89891583e+00  2.89896966e+00  7.46e-13 1.21e-09  2.96e-12  1584s
 206   2.89891632e+00  2.89896594e+00  7.04e-13 1.18e-09  2.73e-12  1593s
 207   2.89891659e+00  2.89896417e+00  6.81e-13 1.14e-09  2.62e-12  1601s
 208   2.89891698e+00  2.89896124e+00  6.73e-13 1.15e-09  2.43e-12  1610s
 209   2.89891740e+00  2.89895843e+00  6.49e-13 1.19e-09  2.26e-12  1618s
 210   2.89891746e+00  2.89895776e+00  6.44e-13 1.16e-09  2.22e-12  1625s
 211   2.89891796e+00  2.89895726e+00 

    4453 DPushes remaining with DInf 1.8099754e+04              2698s
    4343 DPushes remaining with DInf 3.5011755e+04              2711s
    4203 DPushes remaining with DInf 3.4991224e+04              2723s
    4053 DPushes remaining with DInf 1.0131055e+04              2735s
    3923 DPushes remaining with DInf 6.3723358e+03              2748s
    3803 DPushes remaining with DInf 6.3716012e+03              2761s
    3683 DPushes remaining with DInf 6.3712962e+03              2772s
    3553 DPushes remaining with DInf 5.9588367e+03              2785s
    3423 DPushes remaining with DInf 2.6827152e+04              2798s
    3283 DPushes remaining with DInf 1.9591853e+04              2810s
    3153 DPushes remaining with DInf 1.3583675e+04              2822s
    3033 DPushes remaining with DInf 1.3818557e+04              2834s
    2903 DPushes remaining with DInf 9.3041624e+03              2846s
    2773 DPushes remaining with DInf 4.5997950e+03              2858s
    2643 DPushes rem

    1340 DPushes remaining with DInf 1.2629458e+03              4424s
    1220 DPushes remaining with DInf 1.6191207e+03              4439s
    1100 DPushes remaining with DInf 1.3413464e+03              4455s
     970 DPushes remaining with DInf 1.8160722e+03              4472s
     850 DPushes remaining with DInf 1.8083216e+03              4489s
     730 DPushes remaining with DInf 1.6501335e+03              4507s
     610 DPushes remaining with DInf 2.0600531e+03              4525s
     490 DPushes remaining with DInf 2.9207587e+03              4544s
     380 DPushes remaining with DInf 7.0054404e+07              4562s
     250 DPushes remaining with DInf 5.1084476e+07              4581s
     120 DPushes remaining with DInf 6.6510561e+07              4602s
      20 DPushes remaining with DInf 4.8042910e+07              4624s
       0 DPushes remaining with DInf 4.7605029e+07              4629s

  509534 PPushes remaining with PInf 4.8567279e+02              4630s
  493070 PPushes re

...

 1149233    6.7788214e+01   0.000000e+00   2.330244e+07  55097s
 1149373    6.7791942e+01   0.000000e+00   1.405870e+07  55109s
 1149533    6.7793814e+01   0.000000e+00   1.775576e+09  55122s
 1149693    6.7796446e+01   0.000000e+00   6.799840e+06  55135s
 1149863    6.7803126e+01   0.000000e+00   8.251012e+06  55149s
 1150003    6.7803934e+01   0.000000e+00   1.576876e+07  55163s
 1150133    6.7805407e+01   0.000000e+00   1.119550e+07  55174s
 1150313    6.7808602e+01   0.000000e+00   8.963188e+06  55188s
 1150493    6.7816399e+01   0.000000e+00   4.798090e+07  55204s
 1150653    6.7820481e+01   0.000000e+00   5.267641e+07  55219s
 1150823    6.7824231e+01   0.000000e+00   5.972465e+07  55234s
 1150983    6.7829096e+01   0.000000e+00   1.135064e+08  55246s
 1151143    6.7832983e+01   0.000000e+00   1.574381e+07  55258s
 1151243    6.7834515e+01   0.000000e+00   2.937103e+07  55265s
 1151383    6.7835515e+01   0.000000e+00   1.695676e+07  55282s
 1151573    6.7839650e+01   0.000000e+00

 1168473    6.8264007e+01   0.000000e+00   1.940257e+07  56800s
 1168633    6.8267920e+01   0.000000e+00   1.540399e+08  56815s
 1168803    6.8271232e+01   0.000000e+00   5.594578e+07  56828s
 1168973    6.8274510e+01   0.000000e+00   1.425657e+08  56844s
 1169133    6.8279966e+01   0.000000e+00   2.393375e+07  56858s
 1169303    6.8282326e+01   0.000000e+00   1.293035e+07  56873s
 1169453    6.8285873e+01   0.000000e+00   3.330237e+07  56888s
 1169623    6.8290467e+01   0.000000e+00   9.135803e+06  56906s
 1169783    6.8294031e+01   0.000000e+00   8.546065e+06  56921s
 1169903    6.8296793e+01   0.000000e+00   1.286346e+08  56935s
 1170093    6.8308470e+01   0.000000e+00   8.695718e+06  56950s
 1170273    6.8310714e+01   0.000000e+00   1.128089e+08  56966s
 1170433    6.8313834e+01   0.000000e+00   1.102418e+07  56979s
 1170593    6.8317413e+01   0.000000e+00   2.157664e+07  56997s
 1170753    6.8323253e+01   0.000000e+00   5.973328e+07  57011s
 1170883    6.8324589e+01   0.000000e+00

 1188703    7.1596899e+01   0.000000e+00   1.130417e+07  58559s
 1188843    7.1608841e+01   0.000000e+00   8.056540e+07  58571s
 1189033    7.1623265e+01   0.000000e+00   4.984451e+08  58587s
 1189143    7.1652881e+01   0.000000e+00   2.533738e+08  58593s
 1189243    7.1660368e+01   0.000000e+00   1.878068e+07  58600s
 1189343    7.1663562e+01   0.000000e+00   3.489537e+07  58608s
 1189453    7.1672513e+01   0.000000e+00   3.331752e+08  58618s
 1189653    7.1678646e+01   0.000000e+00   4.186199e+07  58633s
 1189803    7.1689045e+01   0.000000e+00   2.530074e+07  58649s
 1189993    7.1696226e+01   0.000000e+00   6.029127e+07  58667s
 1190133    7.1712609e+01   0.000000e+00   6.101279e+06  58681s
 1190233    7.1718979e+01   0.000000e+00   3.114518e+07  58688s
 1190393    7.1733195e+01   0.000000e+00   9.620120e+06  58703s
 1190503    7.1748335e+01   0.000000e+00   3.260243e+08  58709s
 1190613    7.1766645e+01   0.000000e+00   1.929948e+07  58715s
 1190713    7.1779385e+01   0.000000e+00

 1207253    7.2709223e+01   0.000000e+00   1.216265e+07  60010s
 1207403    7.2712481e+01   0.000000e+00   1.327043e+08  60021s
 1207573    7.2725202e+01   0.000000e+00   3.646564e+08  60036s
 1207733    7.2757451e+01   0.000000e+00   1.153104e+07  60050s
 1207923    7.2766433e+01   0.000000e+00   2.562935e+07  60067s
 1208043    7.2770716e+01   0.000000e+00   3.295214e+07  60075s
 1208143    7.2774234e+01   0.000000e+00   8.236239e+06  60082s
 1208343    7.2780887e+01   0.000000e+00   2.590725e+07  60096s
 1208523    7.2789973e+01   0.000000e+00   1.460362e+07  60107s
 1208703    7.2797721e+01   0.000000e+00   2.273732e+08  60125s
 1208883    7.2804352e+01   0.000000e+00   1.907728e+07  60137s
 1208993    7.2808259e+01   0.000000e+00   1.623634e+07  60145s
 1209143    7.2812787e+01   0.000000e+00   4.344945e+07  60159s
 1209313    7.2817357e+01   0.000000e+00   3.436464e+07  60171s
 1209423    7.2822545e+01   0.000000e+00   1.974797e+07  60178s
 1209523    7.2825520e+01   0.000000e+00

 1226173    7.3589278e+01   0.000000e+00   6.949119e+07  61534s
 1226343    7.3599570e+01   0.000000e+00   9.193095e+06  61552s
 1226443    7.3601654e+01   0.000000e+00   6.159110e+06  61557s
 1226623    7.3617423e+01   0.000000e+00   1.067657e+08  61572s
 1226803    7.3627780e+01   0.000000e+00   1.474601e+07  61586s
 1226903    7.3630093e+01   0.000000e+00   2.132490e+07  61594s
 1227063    7.3634798e+01   0.000000e+00   1.193427e+08  61611s
 1227233    7.3636992e+01   0.000000e+00   4.164953e+07  61625s
 1227413    7.3641349e+01   0.000000e+00   1.503189e+07  61639s
 1227583    7.3644259e+01   0.000000e+00   2.825812e+07  61653s
 1227703    7.3648810e+01   0.000000e+00   1.218350e+07  61661s
 1227803    7.3651490e+01   0.000000e+00   5.179815e+07  61667s
 1227983    7.3666471e+01   0.000000e+00   6.135170e+07  61681s
 1228093    7.3673762e+01   0.000000e+00   1.410529e+07  61689s
 1228253    7.3683247e+01   0.000000e+00   2.913179e+07  61704s
 1228423    7.3686757e+01   0.000000e+00